__Obiettivo__

Estrarre alcuni metadati relativi ai file PDF contenuti nella cartella _articlesGrobid/_, mediante l'impiego di apposite librerie.

Nuovamente, i dati estratti si suddividono in:
- __DOI__, _Digital Object Identifier_, identificativo univoco di risorse digitali
- __Title__, titolo dell'articolo scientifico
- __Authors__, autore/autori partecipanti alla stesura del paper preso in considerazione
- __Abstract__, piccolo riassunto del documento, privo di interpretazioni o critiche

Il notebook ricalca lo stesso obiettivo delineato in __metadata.ipynb__, ma prendendo in esame un bacino molto più elevato rispetto ai dieci file contenuti nella cartella _articles/_.
Proprio per questa principale ragione, è stata adeguata una specifica libreria denominata __Grobid__, acronimo di _Generation Of Bibliographic Data_.

_Grobid_ è una libreria di _Machine Learning_, il cui scopo consiste nell'estrazione e conversione di un file PDF in un _formato struttutato XML_. Le funzionalità sviluppate permettono di acquisire un insieme di dati già confezionato, disposti secondo l'ordine gerarchico espresso dal _markup language_ in questione. 

Infatti, come presentato negli snippet di codice successivi, è stata adeguata la libreria _BeatifulSoupe_ affinchè il risultato ottenuto da _Grobid_, memorizzato all'interno di un'apposita directory, fosse convertito in un formato idoneo al linguaggio di programmazione utilizzato.

In [40]:
pdf_path = "../../articles/articlesCiancarini/"

input_dir = "../../articles/articlesGrobid/"
output_dir = "../../articles/articlesTEI/"

Le due classi presentate sono utilizzate rispettivamente per:
- __Author__, classe rappresentativa di tutti gli _autori_ degli articoli scientifici
- __Metadata__, ciascun oggetto istanziato della classe rappresenta i _metadati_ ottenuti di ogni singolo file posto all'interno della cartella _articlesGrobid/_

In [41]:
class Author:
    def __init__(self, forename: str, surname: str):
        self.forename = forename
        self.surname = surname

    def to_unique(self) -> str | None:
        return self.forename + " " + self.surname or None

In [42]:
from typing import List, Dict

class Metadata:
    def __init__(self, DOI: str, path: str, title: str, author: List[Author], keyword: List[str], abstract: str, introduction: str):
        self.DOI = DOI
        self.path = path
        self.title = title
        self.author = author
        self.keyword = keyword
        self.abstract = abstract
        self.introduction = introduction

    def get_dict(self) -> Dict[str, Dict[str, str | List[Author] | None]]:
        return {
            self.path: {
                "DOI": self.DOI,
                "Title": self.title,
                "Author": [item.to_unique() for item in self.author],
                "Keyword": self.keyword,
                "Abstract": self.abstract,
                "Introduction": self.introduction
            }
        }

In [43]:
import os
import shutil

# Took a certain range of PDF from the articlesCiancarini to articlesGrobid
def define_grobid_dir(main_path: str):
    for path in os.listdir(main_path):
        year = int(path[:2])

        try:
            if year in range(0, 100) and path.endswith(".pdf"):
                shutil.copy(main_path + path, input_dir)
        except ValueError as e:
            continue

if not os.path.exists(input_dir):
    os.makedirs(input_dir)
    define_grobid_dir(pdf_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Prima di eseguire lo snippet di codice seguente, è necessario abilitare la _Web API_ di _Grobid_. Ciò può avvenire in due modi, suddivisi in:  
- Installazione di _Grobid_ localmente, in cui è richiesta la previa presenza del _JDK (Java Development Kit)_ e di _Gradle_
- Utilizzo di _Docker_ tramite l'_image_ fornita dalla documentazione, necessaria per realizzare il _container_ associato, ossia l'istanza eseguibile della stessa immagine

La scelta progettuale è ricaduta su _Docker_, data l'estrema semplicità garantita dal tool. Di seguito, è riportato il comando necessario per abilitare la _Web API_:
<div align="center">
    <p><b>docker run --rm --init --ulimit core=0 -p 8070:8070 grobid/grobid:0.8.1</b></p>
</div>

Infine, attraverso un qualsiasi _browser_, è possibile accertarsi se il servizio operi correttamente, accedendo alla pagina dedicata alla _console_ di _Grobid_, esposta tramite _localhost:8070_. La risposta ricevuta dal servizio consiste in un insieme di file _XML_, i quali sono memorizzati all'interno della cartella _TEI/_.

A volte _Grobid_ potrebbe restituire _error 503_, ossia un errore interno del server, in questo caso conviene attendere qualche minuto prima di tentare nuovamente di interagire con il servizio, dato che tutti i thread messi a disposizione sono già utilizzati.

In [ ]:
import requests

from grobid_client.grobid_client import GrobidClient

try:
    grobid_client = GrobidClient(config_path="../../json/grobid/config.json")
    grobid_client.process("processFulltextDocument", input_dir, output_dir, n=10)        
except requests.exceptions.ConnectionError as e:
    print("Connection error during Grobid processing: ", e)
except Exception as e:
    print("Error during Grobid processing: ", e)

In [45]:
def define_xml_paths() -> List[str]:
    path_files = []

    for path in os.listdir(output_dir):
        path_files.append(path)

    return path_files

xml_paths = define_xml_paths()

In [46]:
import re

from bs4 import BeautifulSoup

# Extracting content from XML, after defining a list of progressive tags
def extract_content(soup: BeautifulSoup, tags: List[str]) -> str | None:
    try:
        element = soup.find(tags[0])

        for tag in tags[1:]:
            element = element.find(tag)

        return element.contents[0]
    except Exception:
        return None
    
# Function used to detect authors' forename and username by RegEx
def extract_name(expression: str, persName) -> str | None:
    try:
        match = re.search(expression, str(persName))

        return match.group(1)
    except Exception:
        return None

# BeatifulSoup consists in a data structure representing a parsed XML file
def extract_field_from_xml(path_files: List[str]) -> List[Metadata]:
    list_authors: List[Author] = []
    list_metadata: List[Metadata] = []
    
    for path in sorted(path_files):
        try: 
            if path.endswith(".xml"):
                with open(output_dir + path, "r") as content:
                    xml = content.read()

                soup = BeautifulSoup(xml, "xml")
                
                title = extract_content(soup, ["titleStmt", "title"])
                abstract = extract_content(soup, ["profileDesc", "abstract", "p"])
        
                try:
                    introduction = extract_content(soup, ["body", "div", "p"])
                except Exception:
                    introduction = None

                try: 
                    persNames = soup.find("sourceDesc").find("biblStruct").find("analytic").find_all("persName")
                except Exception:
                    persNames = None

                if persNames is not None:
                    for persName in persNames:
                        forename = extract_name(r"<forename\stype=\"first\">(.*?)<\/forename>", persName)
                        surname = extract_name(r"<surname>(.*?)<\/surname>", persName)
                        
                        if None not in (forename, surname):
                            author = Author(forename, surname)
                            list_authors.append(author)

                list_metadata.append(Metadata(DOI=None, path=path, title=title, author=list_authors, keyword=None, abstract=abstract, introduction=introduction))
                list_authors = []
                
            continue
        except Exception as e:
            print("Error during parsing ", path," file: ", e)

    return list_metadata

list_metadata = extract_field_from_xml(xml_paths)

Nonostante l'applicazione di _Grobid_, alcuni PDF sono privi di _metadati_. Pertanto, affinchè l'analisi condotta possa essere estesa ad un numero sempre più vasto di file,
è stata implementata la libreria _pypdf_; quest'ultima, permette di manipolare ogni singola pagina che componga il PDF. A tal proposito, sono acquisite ulteriori informazioni disponibili tramite il field _metadata_.

In [ ]:
from pypdf import PdfReader

def define_authors(str_author: str) -> List[Author]:
    list_authors: List[Metadata] = []
    try:
        authors = re.split(r"\sand\s", str_author)

        for author in authors:
            items = author.split(" ")
            list_authors.append(Author(items[0], items[1]))

        return list_authors
    except Exception:
        return list_authors

def retrieve_missing_metadata(list_metadata: List[Metadata]):
    for metadata in list_metadata:
        if metadata.title is None or len(metadata.author) == 0:
            pdf_path = re.search(r"(^.*)?.grobid", metadata.path).group(1) + ".pdf"

            try:
                _pdfreader = PdfReader(input_dir + pdf_path).metadata
            except Exception:
                pass
             
            title = _pdfreader.title or None
            author = _pdfreader.author or []

            metadata.title = title
            metadata.path = pdf_path
            metadata.author = define_authors(author)

retrieve_missing_metadata(list_metadata)

Il tipo di alcuni metadati potrebbe differire rispetto al _type str_, pertanto è attuato un check che vada a sovrascrivere il contenuto di alcuni field errati. Ciò avviene per ovviare ad eccezioni di conversione della lista di metadati estratti in formato json.

In [48]:
list_fields = ["abstract", "introduction"]

for metadata in list_metadata:
    for field in list_fields:
        if not isinstance (getattr(metadata, field, None) , str):
            setattr(metadata, field, None)

In [61]:
import numpy
import langdetect

from typing import Tuple
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Extracting candidate keywords from the abstract
def extract_candidates_keyword(text: str) -> numpy.ndarray:
    vectorizer = CountVectorizer(ngram_range=(1, 1), stop_words="english").fit([text])

    return vectorizer.get_feature_names_out()

# Converting string type to numerical type 
def define_embeddings(text: str, candidates: numpy.ndarray) -> Tuple[numpy.ndarray, list]:
    model = SentenceTransformer("distilbert-base-nli-mean-tokens")

    text_embeddings = model.encode([text])
    candidates_embeddings = model.encode(candidates)

    return (text_embeddings, candidates_embeddings)

for metadata in list_metadata:
    try:
        if metadata.abstract is not None and langdetect.detect(metadata.abstract) == "en":
            candidates = extract_candidates_keyword(metadata.abstract)
            paper_embeddings, candidates_embeddings = define_embeddings(metadata.abstract, candidates)

            distances = cosine_similarity(paper_embeddings, candidates_embeddings)
            keywords = [candidates[index] for index in distances.argsort()[0][-5:]]

            metadata.keyword = keywords
    except Exception: 
        pass

In [66]:
import time
import requests

from crossref.restful import Works
from difflib import SequenceMatcher

def get_authors_string(list_authors: List[Author]) -> List[str]:
    str_authors = ""
    for i in range(0, len(list_authors)):
        if i >= len(list_authors):
            str_authors += list_authors[i].forename + " " + list_authors[i].surname
        else:
            str_authors += list_authors[i].forename + " " + list_authors[i].surname + " "

    return str_authors

def similar(str_1: str, str_2: str) -> float:
    return SequenceMatcher(None, str_1, str_2).ratio()

work = Works()

def send_crossref_request(title: str, list_authors: List[str]) -> str:
    url_request = work.query(bibliographic=title, author=get_authors_string(list_authors)).url

    time.sleep(2)

    try:
        response = requests.get(url_request)

        match response.status_code:
            case 200:
                content = response.json()
                message = content.get("message")

                items = message.get("items")
                for item in items:
                    if similar(title, item["title"][0]) > 0.5:
                        return item["DOI"]
                                                                
                    continue
            case 400:
                print("Error during request to CrossRef REST API: Bad Request")
            case _:
                print("Error during request to CrossRef REST API: WTH")
    except Exception as e:
        print("Error during request to CrossRef REST API:", e)

for metadata in list_metadata:
    if None in (metadata.title, metadata.author):
        print("Paper contains null in title or/and author:", metadata.path)
    else:
        doi = send_crossref_request(metadata.title, metadata.author)

        if doi is not None:
            metadata.DOI = doi

Paper contains null in title or/and author: 00Davidson Opponent modeling in Poker.pdf
Error during request to CrossRef REST API: 'title'
Error during request to CrossRef REST API: 'title'
Error during request to CrossRef REST API: 'title'
Paper contains null in title or/and author: 07Ross.pdf
Paper contains null in title or/and author: 08DelliPriscolli.pdf
Error during request to CrossRef REST API: 'title'
Error during request to CrossRef REST API: 'title'
Paper contains null in title or/and author: 09Nippold.pdf
Paper contains null in title or/and author: 10Jerz.pdf
Paper contains null in title or/and author: 11.Reingold.Sheridan.EyeMovementsandVisualExpertiseinChessandMediciner.pdf
Paper contains null in title or/and author: 12Hauptmann - Thomas Eakinss The Chess Players Replayed The Metropolitan Museum J 47.pdf
Paper contains null in title or/and author: 12Linden.pdf
Paper contains null in title or/and author: 13Jerz.pdf
Paper contains null in title or/and author: 13Rappen - Kinderg

In [67]:
list_not_found_dois = list(filter(lambda metadata: metadata.DOI is None, list_metadata))
list_found_dois = list(filter(lambda metadata: metadata.DOI is not None, list_metadata))

La sezione finale del notebook si concentra su alcuni controlli attuati per accertarsi della __bontà__ dei _Digital Object Identifier_ ricavati. Ciò avviene interrogando nuovamente la _REST API_ di _CrossRef_. Tuttavia, in questa casistica, non sono combinati il _title_ e gli _authors_ estrapolati precedentemente, attraverso apposite librerie, ma è inserito all'interno dell'_URL_ lo stesso _DOI_; in questo modo, è possibile definire il grado di similarità tra le informazioni già in possesso rispetto ai nuovi dati ottenuti dalla risposta dell'_Application Programming Interface_.

Di seguito, sono riportate le operazioni principali del comportamento descritto, suddivise in:
- __Definizione della lista__, lista contenente l'insieme di tutti i _metadati_ riferiti ai PDF contenuti nella directory _articlesGrobid/_
- __Invio della richiesta__, richiesta riferita alla _REST API_ di _CrossRef_, da cui, qualora sia positiva la risposta, saranno ricavati i "nuovi" dati necessari per il confronto
- __Definizione grado di similarità__, tramite la libreria _SequenceMatcher_ è estrapolato il _grado di similarità_ del _metadato_ analizzato, qualora dovesse essere maggiore di _0.6_, dato che si tratta di un valore decimale appartenente all'intervallo _[0, 1]_, è rimosso dalla lista

In [68]:
# Small check to detect the correctness of metadata
def define_wrong_dois(list_metadata: List[Metadata]) -> List[Metadata]:
    # List that will contain the "wrong" DOIs retrieved previously
    list_wrong_dois = list(filter(lambda metadata: metadata.DOI is not None, list_metadata))

    for metadata in list_metadata:
        if metadata.DOI is not None:
            url_request = work.query().url + "/" + metadata.DOI
        
            try:
                response = requests.get(url_request)

                match response.status_code:
                    case 200:
                        content = response.json()

                        try:
                            message = content.get("message")

                            # Another check may concern the authors and the publication date
                            titles = message.get("title")
                            authors = message.get("author")
                
                            for title in titles:
                                if similar(metadata.title, title) > 0.6:
                                    list_wrong_dois.remove(metadata)
                                    break

                                continue

                        except Exception as e:
                            print("Field not found", metadata.path)
                            continue
                    case 400:
                        raise Exception("Error during request to CrossRef REST API: Bad Request")
                    case _:
                        raise Exception("Error during request to CrossRef REST API: WTF")
            except Exception as e:
                print("Error during request to CrossRef REST API:", e)

    return list_wrong_dois

list_wrong_dois = define_wrong_dois(list_metadata)
list_correct_dois = list(filter(lambda metadata: metadata in list_found_dois and metadata not in list_wrong_dois, list_metadata))

In [69]:
import json

def define_list(_list: List[Metadata]) -> List:
    list_json = []

    for item in _list:
        list_json.append(item.get_dict())

    return list_json

with open("../../json/extraction/metadata_completed.json", "w") as file:
    json.dump(define_list(list_metadata), file, indent=3)

In [72]:
import pandas 

dict_doi_summary = {
    "Total": len(list_metadata),
    "Found": len(list_found_dois),
    "Not found": len(list_not_found_dois)
}

dict_doi_accuracy = {
    "Found": len(list_found_dois),
    "Correct": len(list_correct_dois),
    "Wrong": len(list_wrong_dois)
}

df_doi_summary = pandas.DataFrame.from_dict(dict_doi_summary, orient="index", columns=["# DOI"])
display(df_doi_summary)

df_doi_accuracy = pandas.DataFrame.from_dict(dict_doi_accuracy, orient="index", columns=["# DOI"])
display(df_doi_accuracy)

# DOI
Total       1770
Found       1131
Not found    639

# DOI
Found     1131
Correct    944
Wrong      187